In [ ]:
from google.colab import drive
drive.mount('/content/drive')

ValueError: Mountpoint must not already contain files

# ***LIBRARIES***

In [ ]:
import os
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import torchaudio
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
!pip install tensorflow_hub torchaudio pandas

In [ ]:
!pip install tensorflow_hub

# **TIRLLSON**

Logic

In [ ]:
# #Trillson Reference: https://tfhub.dev/google/nonsemantic-speech-benchmark/trillsson4/1
m = hub.KerasLayer('https://tfhub.dev/google/trillsson4/1')
def extract_features(path):
    sample_rate = 16000
    array, fs = torchaudio.load(path)
    array = np.array(array)
    # NOTE: Audio should be floats in [-1, 1], sampled at 16kHz. Model input is of
    # the shape [batch size, time].
    embeddings = m(array)['embedding']
    # Models internally aggregate over time. For a time-series of embeddings, the
    # user can frame audio however they want.
    embeddings.shape.assert_is_compatible_with([None, 1024])
    embeddings = np.squeeze(np.array(embeddings), axis = 0)
    return embeddings

CSV

In [ ]:
# Load the Trillson model from TensorFlow Hub
m = hub.KerasLayer('https://tfhub.dev/google/trillsson4/1')

# Function to extract features from an audio file
def extract_features(path):
    sample_rate = 16000
    array, fs = torchaudio.load(path)
    array = np.array(array)
    # NOTE: Audio should be floats in [-1, 1], sampled at 16kHz. Model input is of
    # the shape [batch size, time].
    embeddings = m(array)['embedding']
    # Models internally aggregate over time. For a time-series of embeddings, the
    # user can frame audio however they want.
    embeddings.shape.assert_is_compatible_with([None, 1024])
    embeddings = np.squeeze(np.array(embeddings), axis=0)
    return embeddings

# Function to save features to a CSV file
def save_to_csv(features, output_file):
    df = pd.DataFrame(features)
    df.to_csv(output_file, index=False)
    print(f"Features saved to {output_file}")

# Example usage
input_audio_file = '/content/drive/MyDrive/Autistic/Combined/Abnormal/A001.wav'
output_csv_file = 'file.csv'

# Extract features from the audio file
features = extract_features(input_audio_file)

# Save features to a CSV file
save_to_csv(features, output_csv_file)


Features saved to file.csv


ALL features

In [ ]:
# Load the Trill model from TensorFlow Hub
model = hub.load('https://tfhub.dev/google/nonsemantic-speech-benchmark/trill/3')

def extract_features(audio_path):
    # Load the audio file
    audio, sr = librosa.load(audio_path, sr=None)

    # Resample the audio to 16kHz
    if sr != 16000:
        audio = librosa.resample(audio, orig_sr=sr, target_sr=16000)
        sr = 16000

    # Convert the audio waveform to the format expected by the model
    audio = audio[np.newaxis, :]
    audio = tf.convert_to_tensor(audio, dtype=tf.float32)

    # Extract features
    embeddings = model(samples=audio, sample_rate=sr)

    # Handle the dictionary output
    # Replace 'embedding' with the correct key if different
    embedding_key = 'embedding' if 'embedding' in embeddings else list(embeddings.keys())[0]
    return embeddings[embedding_key].numpy()

# Example usage
audio_path = '/content/drive/MyDrive/Autistic/Combined/Abnormal/A001.wav'  # Replace with your audio file path
features = extract_features(audio_path)

# Save the features to a CSV file
csv_file = 'extracted_features.csv'  # Output CSV file name

# Flatten the features array for CSV writing
flattened_features = features.flatten()

with open(csv_file, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(flattened_features)

print(f"Features saved to {csv_file}")


Features saved to extracted_features.csv


FOlder

In [ ]:
m = hub.KerasLayer('https://tfhub.dev/google/trillsson4/1')

# Function to extract features from an audio file
def extract_features(path):
    sample_rate = 16000
    array, fs = torchaudio.load(path)
    array = np.array(array)
    # NOTE: Audio should be floats in [-1, 1], sampled at 16kHz. Model input is of
    # the shape [batch size, time].
    embeddings = m(array)['embedding']
    # Models internally aggregate over time. For a time-series of embeddings, the
    # user can frame audio however they want.
    embemddings.shape.assert_is_compatible_with([None, 1024])
    embeddings = np.squeeze(np.array(embeddings), axis=0)
    return embeddings

# Function to save features to a CSV file
def save_to_csv(features, file_names, output_file):
    df = pd.DataFrame(features, columns=[f'feature_{i}' for i in range(features.shape[1])])
    df['file_name'] = file_names
    df.to_csv(output_file, index=False)
    print(f"Features saved to {output_file}")

# Function to process all audio files in a folder
def process_folder(input_folder, output_csv_file):
    features_list = []
    file_names = []

    for filename in os.listdir(input_folder):
        if filename.endswith(".wav"):
            file_path = os.path.join(input_folder, filename)
            features = extract_features(file_path)
            features_list.append(features)
            file_names.append(filename)

    features_array = np.vstack(features_list)
    save_to_csv(features_array, file_names, output_csv_file)

# Example usage
input_audio_folder = '/content/drive/MyDrive/Autistic/Combined/Abnormal'
output_csv_file = 'Abnormal.csv'

# Process all audio files in the folder and save features to a CSV file
process_folder(input_audio_folder, output_csv_file)


In [ ]:
#   seprate csv

In [ ]:
# Load the Trillson model from TensorFlow Hub
m = hub.KerasLayer('https://tfhub.dev/google/trillsson4/1')

# Function to extract features from an audio file
def extract_features(path):
    sample_rate = 16000
    array, fs = torchaudio.load(path)

    # Check if the sampling rate is not 16 kHz
    if fs != sample_rate:
        # Resample audio to 16 kHz
        resampler = torchaudio.transforms.Resample(fs, sample_rate)
        array = resampler(array)
        print(f"Resampled audio to {sample_rate} Hz")

    array = np.array(array)

    # Ensure the waveform has only one channel (mono)
    if array.shape[0] > 1:
        array = array.mean(dim=0, keepdim=True)

    embeddings = m(array)['embedding']
    embeddings.shape.assert_is_compatible_with([None, 1024])

    # Squeeze dimensions appropriately
    embeddings = np.squeeze(np.array(embeddings), axis=0)

    return embeddings

# Function to save features to a CSV file
def save_to_csv(features, output_file):
    df = pd.DataFrame(features)
    df.to_csv(output_file, index=False)
    print(f"Features saved to {output_file}")

# Function to process a folder of audio files
def process_folder(input_folder, output_folder):
    os.makedirs(output_folder, exist_ok=True)

    for filename in os.listdir(input_folder):
        if filename.endswith('.wav'):
            input_audio_file = os.path.join(input_folder, filename)
            output_csv_file = os.path.join(output_folder, f"{os.path.splitext(filename)[0]}.csv")

            # Check if the CSV file already exists for the current audio file
            if not os.path.exists(output_csv_file):
                features = extract_features(input_audio_file)
                save_to_csv(features, output_csv_file)
            else:
                print(f"Features already exist for {input_audio_file}. Skipping.")

# Example usage
input_folder = '/content/drive/MyDrive/Autistic/Combined/Abnormal'
output_folder = '/content/drive/MyDrive/Autistic/Combined/Features_trillson/Abnormal'

process_folder(input_folder, output_folder)


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/Autistic/Combined/Abnormal'